## SDLC Use Case Generation

In [123]:
import argparse
import os
from pydantic import BaseModel, Field, ValidationError
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from groq import Groq
from langchain_groq import ChatGroq
from IPython.display import Markdown
from langchain_core.messages import HumanMessage
import itertools


In [124]:
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [125]:
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY environment variable not set.")
client = Groq(api_key=groq_api_key)


In [126]:
uCounter = itertools.count(1)
fCounter = itertools.count(1)
us_id = f"{next(uCounter):03d}"
fr_id = f"{next(uCounter):03d}"
UserStory_id = f"US-{us_id}"
FR_id = f"FR-{us_id}-{fr_id}"

uids = (f"{i:03d}" for i in range(1, 100))
fids = (f"{i:03d}" for i in range(1, 100))
uids_next = f"{next(uids)}"
fids_next = f"{next(fids)}"
story_id = f"US{uids_next}"
func_id = f"FR-{story_id}-{fids_next}"
print(story_id)
print(func_id)


US001
FR-US001-001


In [133]:
user_stories_system_prompt = """
** Role & Objective **

You are an expert Agile Product Owner specializing in crafting clear, concise, and actionable User Stories.  
Your task is to analyze the provided `user_story` and `user_feedback`, identify gaps or ambiguities, and refine or enhance the User Stories accordingly.  
Return the final output strictly in the specified JSON format.

---

**TASK BREAKDOWN:**

1. Carefully analyze the input `user_story` and `user_feedback`.  
2. Identify missing information, unclear language, or inconsistencies.  
3. Refine the User Stories to ensure they follow the standard format:  
   - "As a [user], I want [goal], so that [value]."  
4. Write **2 to 4** clear, specific, and testable acceptance criteria per story.  
5. Maintain alignment with business goals and user needs.  
6. Avoid technical implementation details or vague language.  
7. Ensure each User Story is independent, coherent, and ready for development.

---

**EXPECTED OUTPUT FORMAT (JSON array):**

```json
[
  {
    "story_id": "{story_id}",
    "title": "Customer Allocation - Item Substitution Scenario - IteM",
    "description": "Manage the allocation and substitution of items based on availability and allocation rules to ensure correct order quantities.",
    "acceptance_criteria": [
      {
        "{story_id}-{func_id}": "Does original item have substitutes?",
        "{story_id}-{func_id}": "If yes, check sub rank 1 first."
      }
    ]
  }
]

**GUIDELINES:**

- Ensure each User Story is clear, independent, and testable.
- Incorporate user feedback without losing sight of core business objectives.
- Acceptance criteria must be between 2 and 4 concise, measurable points.
- Avoid technical jargon, implementation details, or vague requirements.
- Use consistent and professional language suitable for Agile teams.
- Respond only with the JSON output as specified. Do not include explanations or additional text. 


**Enhancements made:**

- Clarified role and task objectives.  
- Emphasized standard user story format and acceptance criteria quality.  
- Added instructions to avoid technical details and vague language.  
- Specified strict output format and response behavior.  
- Improved formatting and readability for easier parsing by LLMs.


"""


In [134]:
project_title = "LeeSar: Your End-to-End Order Management Solution"
project_description = "LeeSar is a comprehensive Order Management application that allows users to perform seamless transactions using the Fusion Order Management (FOM). Beyond Basic Order, it offers Extensions on the Orders that are created."
requirements = [
    "Does original item have substitutes?"
]
requirements_string = "\n".join(requirements)
requirements_string

'Does original item have substitutes?'

In [135]:
# llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.1-8b-instant",
    temperature=0.5
)

# parser = JsonOutputParser(pydantic_object=Products)
parser = JsonOutputParser()
format = parser.get_format_instructions()


In [136]:
prompt = PromptTemplate(
    template="{system_prompt} \n {format_instruction} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
    partial_variables={"format_instruction" : parser.get_format_instructions()}
)
#prompt.model_dump_json(indent=2)

In [137]:
def generate_user_story_from_llm(project_title, project_description, user_story_requirements):
    user_query = f"Create User Stories for the Scenario for the \nProject Title: {project_title}, \nProject Description: {project_description} and \nUser Story Requirement: {user_story_requirements}"
    chain = prompt | llm | parser
    invoke_input = {"system_prompt" : user_stories_system_prompt, "human_query" : user_query}
    response = chain.invoke(invoke_input)
    #Markdown(response)
    #print(response)
    return response



In [138]:
user_stories = generate_user_story_from_llm(project_title, project_description, requirements_string)
user_stories

[{'story_id': 'LS-001',
  'title': 'Customer Allocation - Item Substitution Scenario',
  'description': 'As a customer service representative, I want to check if an original item has substitutes, so that I can provide the customer with alternatives if the item is out of stock or not available.',
  'acceptance_criteria': [{'LS-001-AC1': 'The system displays a list of substitutes for the original item when it is out of stock or not available.',
    'LS-001-AC2': 'The substitutes are ranked based on their availability and allocation rules.',
    'LS-001-AC3': 'The system alerts the customer service representative if no substitutes are available for the original item.'}]}]